In [3]:
import pandas as pd
import pandas_gbq as pd_gbq
import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas as pd
import numpy as np

/Users/ivan-can.arisoy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [36]:
client = bigquery.Client(project="teads-exploration")

sql = """
WITH flattened_data AS (
  SELECT
    t.country_iso_code,
    t.household_id,
    t.created_at,
    vid.vid,
    vid.env,
    vid.ip_hash,
    vid.connection_type,
    vid.browsers,
    vid.ua_devices,
    vid.cities,
    device.standardised_name as device_standardised_name,
    viewer.name as viewer_name,
    viewer.value as viewer_value
  FROM `teads-exploration.tmp_30d.ivan_shared_ids_study_2`  t
  CROSS JOIN UNNEST(t.vids) as vid
  LEFT JOIN UNNEST(vid.device_infos) as device
  LEFT JOIN UNNEST(vid.shared_viewer_ids) as viewer
),
filtered_data AS (
  SELECT DISTINCT
    country_iso_code,
    household_id,
    created_at,
    vid,
    env,
    ip_hash,
    connection_type,
    browsers,
    ua_devices,
    cities,
    device_standardised_name,
    viewer_name,
    viewer_value
  FROM flattened_data
  WHERE NOT EXISTS (
    SELECT 1 
    FROM flattened_data f2 
    WHERE f2.household_id = flattened_data.household_id 
    AND f2.vid = flattened_data.vid
    AND f2.viewer_name = 'id5Id'
  )
)
SELECT *
FROM filtered_data
ORDER BY household_id, created_at, vid;
"""

df_query = client.query(sql).to_dataframe(bqstorage_client=bigquery_storage.BigQueryReadClient())

df_query.head()

,country_iso_code,household_id,created_at,vid,env,ip_hash,connection_type,browsers,ua_devices,cities,device_standardised_name,viewer_name,viewer_value
0,US,-9223276717646836784,2025-01-18 11:00:00+00:00,537bf256-6415-4e60-8170-ed2ddcf76027,web,[768689590813166942],[wifi],[Chrome],[Phone],[buffalo],Unknown Generic Android Tablet,None,None
1,US,-9223276717646836784,2025-01-18 11:00:00+00:00,acda0e07-5622-40f4-9597-c11814e3d4b4,web,"[768689590813166942, 4484783861805880557]","[mobile, wifi]",[Chrome Mobile],[Phone],"[buffalo, geneva]",Samsung Galaxy S21 Ultra 5G,None,None
2,US,-9223276717646836784,2025-01-18 11:00:00+00:00,acda0e07-5622-40f4-9597-c11814e3d4b4,web,"[768689590813166942, 4484783861805880557]","[mobile, wifi]",[Chrome Mobile],[Phone],"[buffalo, geneva]",Unknown Generic Android Mobile,None,None
3,US,-9223276229937712256,2025-01-14 00:00:00+00:00,6652f0f0-4450-5585-8bc8-6f2675711280,ctv,[4710804623793478939],[mobile],[unknown],[IPTV],[houston],Roku Digital Video player,None,None
4,US,-9223169821723962225,2025-01-13 15:00:00+00:00,668e9add-16f5-ba55-d6bd-b7c754248369,ctv,[2461993606044529373],[wifi],[unknown],[IPTV],[los angeles],Samsung Tizen TV,None,None


In [71]:
df_query

,country_iso_code,household_id,created_at,vid,env,ip_hash,connection_type,browsers,ua_devices,cities,device_standardised_name,viewer_name,viewer_value
0,US,-9223276717646836784,2025-01-18 11:00:00+00:00,537bf256-6415-4e60-8170-ed2ddcf76027,web,[768689590813166942],[wifi],[Chrome],[Phone],[buffalo],Unknown Generic Android Tablet,None,None
1,US,-9223276717646836784,2025-01-18 11:00:00+00:00,acda0e07-5622-40f4-9597-c11814e3d4b4,web,"[768689590813166942, 4484783861805880557]","[mobile, wifi]",[Chrome Mobile],[Phone],"[buffalo, geneva]",Samsung Galaxy S21 Ultra 5G,None,None
2,US,-9223276717646836784,2025-01-18 11:00:00+00:00,acda0e07-5622-40f4-9597-c11814e3d4b4,web,"[768689590813166942, 4484783861805880557]","[mobile, wifi]",[Chrome Mobile],[Phone],"[buffalo, geneva]",Unknown Generic Android Mobile,None,None
3,US,-9223276229937712256,2025-01-14 00:00:00+00:00,6652f0f0-4450-5585-8bc8-6f2675711280,ctv,[4710804623793478939],[mobile],[unknown],[IPTV],[houston],Roku Digital Video player,None,None
4,US,-9223169821723962225,2025-01-13 15:00:00+00:00,668e9add-16f5-ba55-d6bd-b7c754248369,ctv,[2461993606044529373],[wifi],[unknown],[IPTV],[los angeles],Samsung Tizen TV,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814403,US,9223348700142645532,2025-01-12 22:00:00+00:00,0d1b2c2e-f8d1-4daf-ae72-469e6d0e60de,web,[-1286731533017783601],[wifi],[Silk],[Tablet],[virginia beach],Amazon Fire HD 10,None,None
1814404,US,9223352914900239166,2025-01-14 08:00:00+00:00,dc1fb582-6bdb-563e-8df0-48bd7fb90e1b,ctv,[-5520468168907918995],[wifi],[unknown],[IPTV],[springdale],Roku Digital Video player,None,None
1814405,US,9223352914900239166,2025-01-14 08:00:00+00:00,ff4a717c-41e3-4424-b1b4-f24eda6169ac,web,[-7342105050975757126],[wired],[Chrome Mobile],[Phone],[kansas city],Unknown Generic Android Mobile,None,None
1814406,US,9223358029016505633,2025-01-12 23:00:00+00:00,bfd1ee10-27bb-4aaf-abd3-9fd7ea4d51be,inapp,"[3377951229266479113, -3228744965884679268, 45...",[wifi],"[Safari mobile, Android WebView]",[Phone],"[riverside, south jordan, salt lake city]",Samsung Galaxy Z Fold 3,None,None


In [37]:
web_counts = df_query[df_query['env'] == 'web']['household_id'].value_counts()
valid_households = web_counts[web_counts >= 3].index

# Then filter the original dataframe to keep only those household_ids
filtered_df = df_query[df_query['household_id'].isin(valid_households)]

In [ ]:
web_counts = df_query[df_query['env'] == 'web']['household_id'].value_counts()
valid_households = web_counts[web_counts >= 1].index

# Then filter the original dataframe to keep only those household_ids
filtered_df = df_query[df_query['household_id'].isin(valid_households)]

In [38]:
filtered_df = filtered_df.rename(columns={'device_standardised_name': 'standardised_name'})

In [52]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 673709 entries, 0 to 1814402
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   country_iso_code   673709 non-null  object             
 1   household_id       673709 non-null  Int64              
 2   created_at         673709 non-null  datetime64[ns, UTC]
 3   vid                673709 non-null  object             
 4   env                673709 non-null  object             
 5   ip_hash            673709 non-null  object             
 6   connection_type    673709 non-null  object             
 7   browsers           673709 non-null  object             
 8   ua_devices         673709 non-null  object             
 9   cities             673709 non-null  object             
 10  standardised_name  512242 non-null  object             
 11  viewer_name        56306 non-null   object             
 12  viewer_value       56306 non-

In [53]:
k25 = filtered_df['household_id'].drop_duplicates().sample(n=5000, random_state=42)

# Filter the dataframe to keep only those household IDs
df = filtered_df[filtered_df['household_id'].isin(k25)]

In [54]:
household_user_edges = df[['created_at', 'household_id', 'viewer_value']].drop_duplicates()
household_user_edges = household_user_edges.rename(columns={'household_id': 'source', 'viewer_value': 'target'})
household_user_edges['relationship'] = 'Household-User'
household_user_edges

,created_at,source,target,relationship
2170,2025-01-16 12:00:00+00:00,-9200385283866435985,None,Household-User
2172,2025-01-16 12:00:00+00:00,-9200385283866435985,d3776100dbce5e8f31fad5ddd3f516d53938a4c8273cdd...,Household-User
2798,2025-01-12 21:00:00+00:00,-9192807107845688163,None,Household-User
3020,2025-01-19 03:00:00+00:00,-9190073361424575441,None,Household-User
3066,2025-01-13 04:00:00+00:00,-9189549264388806478,None,Household-User
...,...,...,...,...
1812555,2025-01-15 23:00:00+00:00,9204910166019323690,None,Household-User
1812976,2025-01-14 20:00:00+00:00,9209590246548167742,None,Household-User
1813153,2025-01-12 21:00:00+00:00,9211519178587349327,None,Household-User
1813289,2025-01-14 00:00:00+00:00,9212915449340000494,None,Household-User


In [55]:
# User to Device connections
user_device_edges = df[['created_at', 'viewer_value', 'vid']].drop_duplicates()
user_device_edges = user_device_edges.rename(columns={'viewer_value': 'source', 'vid': 'target'})
user_device_edges['relationship'] = 'User-Device'

user_device_edges

,created_at,source,target,relationship
2170,2025-01-16 12:00:00+00:00,None,9b506c98-8d12-4077-9e8d-5dcf2373eae2,User-Device
2171,2025-01-16 12:00:00+00:00,None,d5955aae-3d3c-ae7c-1cdd-f3540bc022cb,User-Device
2172,2025-01-16 12:00:00+00:00,d3776100dbce5e8f31fad5ddd3f516d53938a4c8273cdd...,da4f4d14-0749-44ce-8e44-429a302d501c,User-Device
2798,2025-01-12 21:00:00+00:00,None,3b0d3e16-63e4-4dc8-a4a2-4f402f61afb4,User-Device
2799,2025-01-12 21:00:00+00:00,None,7b642f26-8f94-4cd5-9c7f-bfd09ef15faf,User-Device
...,...,...,...,...
1814133,2025-01-13 18:00:00+00:00,None,9f4a33b1-236c-414b-b7a9-c0d24b500b20,User-Device
1814134,2025-01-13 18:00:00+00:00,None,a390eeaf-7d6c-4c93-95e5-32ed1dec60da,User-Device
1814135,2025-01-13 18:00:00+00:00,None,b9ff9176-a1ad-4ce5-87e8-6640be3aded7,User-Device
1814136,2025-01-13 18:00:00+00:00,None,c2d80cdc-d62c-450a-a6c8-90ba437e962d,User-Device


In [56]:
id_vids = set(df[df['viewer_name'].notna()]['vid'].unique())
household_device_df = df[(df['viewer_name'].isna()) | (~df['vid'].isin(id_vids))]

# Household to Device connections
household_device_edges = household_device_df[['created_at', 'household_id', 'vid']].drop_duplicates()
household_device_edges = household_device_edges.rename(columns={'household_id': 'source', 'vid': 'target'})
household_device_edges['relationship'] = 'Household-Device'
household_device_edges

,created_at,source,target,relationship
2170,2025-01-16 12:00:00+00:00,-9200385283866435985,9b506c98-8d12-4077-9e8d-5dcf2373eae2,Household-Device
2171,2025-01-16 12:00:00+00:00,-9200385283866435985,d5955aae-3d3c-ae7c-1cdd-f3540bc022cb,Household-Device
2798,2025-01-12 21:00:00+00:00,-9192807107845688163,3b0d3e16-63e4-4dc8-a4a2-4f402f61afb4,Household-Device
2799,2025-01-12 21:00:00+00:00,-9192807107845688163,7b642f26-8f94-4cd5-9c7f-bfd09ef15faf,Household-Device
2800,2025-01-12 21:00:00+00:00,-9192807107845688163,929573d7-98e9-47ba-9dc1-f3ffd736963b,Household-Device
...,...,...,...,...
1814133,2025-01-13 18:00:00+00:00,9220750467488575936,9f4a33b1-236c-414b-b7a9-c0d24b500b20,Household-Device
1814134,2025-01-13 18:00:00+00:00,9220750467488575936,a390eeaf-7d6c-4c93-95e5-32ed1dec60da,Household-Device
1814135,2025-01-13 18:00:00+00:00,9220750467488575936,b9ff9176-a1ad-4ce5-87e8-6640be3aded7,Household-Device
1814136,2025-01-13 18:00:00+00:00,9220750467488575936,c2d80cdc-d62c-450a-a6c8-90ba437e962d,Household-Device


In [57]:
# Combine all edge DataFrames
edges = pd.concat([household_user_edges, user_device_edges, household_device_edges], ignore_index=True)

In [58]:
# drop NaN values
edges = edges.dropna()

In [59]:
#rename created_at to time
edges = edges.rename(columns={'created_at': 'time'})
edges

,time,source,target,relationship
1,2025-01-16 12:00:00+00:00,-9200385283866435985,d3776100dbce5e8f31fad5ddd3f516d53938a4c8273cdd...,Household-User
6,2025-01-12 21:00:00+00:00,-9183468325073019856,baf4eabe34c7dd36f282439d53bf4945a7021f64942feb...,Household-User
16,2025-01-14 23:00:00+00:00,-9131635824071316309,6fc692a208786bad7eafce61841a16d53938dfb7e09739...,Household-User
17,2025-01-14 23:00:00+00:00,-9131635824071316309,ec185GRlBi9duUkClCt7IkNtO4UTUVh_GrvSS3iz88XGuz...,Household-User
18,2025-01-14 23:00:00+00:00,-9131635824071316309,a7282af97dee60a21625e16a5603a9fb927a91828f7e97...,Household-User
...,...,...,...,...
83670,2025-01-13 18:00:00+00:00,9220750467488575936,9f4a33b1-236c-414b-b7a9-c0d24b500b20,Household-Device
83671,2025-01-13 18:00:00+00:00,9220750467488575936,a390eeaf-7d6c-4c93-95e5-32ed1dec60da,Household-Device
83672,2025-01-13 18:00:00+00:00,9220750467488575936,b9ff9176-a1ad-4ce5-87e8-6640be3aded7,Household-Device
83673,2025-01-13 18:00:00+00:00,9220750467488575936,c2d80cdc-d62c-450a-a6c8-90ba437e962d,Household-Device


In [60]:
print("Sample of connection_type:", df['connection_type'].iloc[0])
print("Type:", type(df['connection_type'].iloc[0]))


Sample of connection_type: ['wifi']
Type: <class 'numpy.ndarray'>


In [61]:
df_processed = df.copy()

# Columns to convert to string
columns_to_convert = ['env', 'ip_hash', 'connection_type', 'browsers', 'ua_devices', 'cities']


for col in columns_to_convert:
    # Convert lists/arrays to a formatted string representation with parentheses
    df_processed[col] = df_processed[col].apply(lambda x: f"( {' ; '.join(map(str, x))} )" if isinstance(x, (list, np.ndarray)) else str(x))

# Now create the household metadata
household_metadata = df_processed[['household_id', 'env', 'ip_hash', 'browsers', 'connection_type', 'ua_devices', 'cities', 'standardised_name']].drop_duplicates()
household_metadata = household_metadata.rename(columns={'household_id': 'id'})

# Set the columns as specified
household_metadata['env'] = 'Household'
household_metadata['browsers'] = '-'
household_metadata['connection_type'] = '-'
household_metadata['ua_devices'] = 'Household'
household_metadata['standardised_name'] = '-'
household_metadata = household_metadata.drop_duplicates()


In [62]:
# Calculate number of users per household from edges dataset
household_nb_of_users = edges[edges['relationship'] == 'Household-User'].groupby('source')['target'].nunique().reset_index()
household_nb_of_users.rename(columns={'source': 'id', 'target': 'household_nb_of_users'}, inplace=True)

# Calculate number of VIDs per household from df dataset
household_nb_of_vids = df.groupby('household_id')['vid'].nunique().reset_index()
household_nb_of_vids.rename(columns={'household_id': 'id', 'vid': 'household_nb_of_vids'}, inplace=True)

# Fill NaN values with 0 and convert to integer
household_nb_of_users['household_nb_of_users'] = household_nb_of_users['household_nb_of_users'].fillna(0).astype(int)
household_nb_of_vids['household_nb_of_vids'] = household_nb_of_vids['household_nb_of_vids'].fillna(0).astype(int)

In [63]:


# Metadata for Users (e.g., id5Id, criteoId) df[['household_id', 'env', 'ip_hash', 'browsers', 'connection_type', 'ua_devices', 'cities', 'device_standardised_name']].drop_duplicates()
user_metadata = df_processed[['viewer_value', 'env', 'ip_hash', 'browsers', 'connection_type', 'ua_devices', 'cities', 'standardised_name']].drop_duplicates()
user_metadata = user_metadata.rename(columns={'viewer_value': 'id'})
user_metadata['ua_devices'] = 'User: PanoramaID'
user_metadata['browsers'] = '-'
user_metadata['standardised_name'] = '-'
user_metadata

user_metadata = user_metadata.drop_duplicates()
user_metadata.dropna(inplace=True)
user_metadata

,id,env,ip_hash,browsers,connection_type,ua_devices,cities,standardised_name
2172,d3776100dbce5e8f31fad5ddd3f516d53938a4c8273cdd...,web,( 8580074699794839955 ),-,( wifi ),User: PanoramaID,( memphis ),-
3619,baf4eabe34c7dd36f282439d53bf4945a7021f64942feb...,web,( -5157660449457695196 ),-,( wifi ),User: PanoramaID,( shady side ),-
8955,6fc692a208786bad7eafce61841a16d53938dfb7e09739...,web,( 1228620433842065099 ; -8677201957402249987 ;...,-,( wifi ),User: PanoramaID,( laurel ; annapolis ),-
8956,ec185GRlBi9duUkClCt7IkNtO4UTUVh_GrvSS3iz88XGuz...,web,( 1228620433842065099 ; -8677201957402249987 ;...,-,( wifi ),User: PanoramaID,( laurel ; annapolis ),-
8961,a7282af97dee60a21625e16a5603a9fb927a91828f7e97...,web,( 3309095672985860527 ),-,( wifi ),User: PanoramaID,( laurel ),-
...,...,...,...,...,...,...,...,...
1800569,a20c7d23b05153ec21c0f6f3c9d516d539380ef3e6655c...,web,( 6401984888606647531 ),-,( mobile ),User: PanoramaID,( lake oswego ),-
1801335,2bc81732125fef69f098927d744b4945a7021d9477b976...,web,( 4282356585966481271 ),-,( wifi ),User: PanoramaID,( dallas ),-
1804026,5ef9fa59eed0e62b276f7eb5231e4945a702cc2e9657b1...,web,( 6075797506074403027 ),-,( wifi ),User: PanoramaID,( san diego ),-
1808776,85c9cd8d9af2da2531798253987616d539381a71b94362...,web,( -8776728284709514475 ; 20187293744560087 ),-,( wifi ),User: PanoramaID,( blaine ; seattle ),-


In [64]:
user_nb_of_housheolds = df.groupby('viewer_value')['household_id'].nunique().reset_index()
user_nb_of_housheolds.rename(columns={'viewer_value': 'id', 'household_id': 'user_nb_of_households'}, inplace=True)

user_nb_of_vids = df.groupby('viewer_value')['vid'].nunique().reset_index()
user_nb_of_vids.rename(columns={'viewer_value': 'id', 'vid': 'user_nb_of_vids'}, inplace=True)


In [65]:
# Metadata for Devices (VIDs)
device_metadata = df_processed[['vid',  'env', 'ip_hash', 'browsers', 'connection_type', 'ua_devices', 'cities', 'standardised_name']].drop_duplicates()
device_metadata = device_metadata.rename(columns={'vid': 'id'})
device_metadata = device_metadata.drop_duplicates()

# Combine all metadata
metadata = pd.concat([household_metadata, user_metadata, device_metadata], ignore_index=True).drop_duplicates(subset='id').reset_index(drop=True)
device_metadata

,id,env,ip_hash,browsers,connection_type,ua_devices,cities,standardised_name
2170,9b506c98-8d12-4077-9e8d-5dcf2373eae2,web,( 4347902281762601649 ),( Android WebView ),( wifi ),( Phone ),( memphis ),Samsung Galaxy S21 5G Enterprise Edition
2171,d5955aae-3d3c-ae7c-1cdd-f3540bc022cb,web,( 8580074699794839955 ),( Safari ),( wifi ),( PC ),( memphis ),None
2172,da4f4d14-0749-44ce-8e44-429a302d501c,web,( 8580074699794839955 ),( Chrome ),( wifi ),( PC ),( memphis ),None
2798,3b0d3e16-63e4-4dc8-a4a2-4f402f61afb4,web,( 3925892373630619020 ),( Android WebView ),( wifi ),( Phone ),( johnstown ),Cricket Outlast 5G
2799,7b642f26-8f94-4cd5-9c7f-bfd09ef15faf,web,( 3925892373630619020 ),( Android WebView ),( wifi ),( Phone ),( johnstown ),Cricket Outlast 5G
...,...,...,...,...,...,...,...,...
1814133,9f4a33b1-236c-414b-b7a9-c0d24b500b20,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+
1814134,a390eeaf-7d6c-4c93-95e5-32ed1dec60da,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+
1814135,b9ff9176-a1ad-4ce5-87e8-6640be3aded7,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+
1814136,c2d80cdc-d62c-450a-a6c8-90ba437e962d,web,( -628032056789376465 ),( Chrome ),( wifi ),( Phone ),( hendersonville ),Unknown Generic Android Tablet


In [66]:
#number of housheolds associated
vid_nb_of_housheolds = df.groupby('vid')['household_id'].nunique().reset_index()
vid_nb_of_housheolds.rename(columns={'vid': 'id', 'household_id': 'vid_nb_of_households'}, inplace=True)


In [67]:
vid_nb_of_users = df.groupby('vid')['viewer_value'].nunique().reset_index()
vid_nb_of_users.rename(columns={'vid': 'id', 'viewer_value': 'vid_nb_of_users'}, inplace=True)



In [68]:
metadata

,id,env,ip_hash,browsers,connection_type,ua_devices,cities,standardised_name
0,-9200385283866435985,Household,( 4347902281762601649 ),-,-,Household,( memphis ),-
1,-9192807107845688163,Household,( 3925892373630619020 ),-,-,Household,( johnstown ),-
2,-9190073361424575441,Household,( -6351207366429801774 ),-,-,Household,( ft lee ),-
3,-9189549264388806478,Household,( 2816271962527781015 ),-,-,Household,( columbus ),-
4,-9183468325073019856,Household,( -5157660449457695196 ),-,-,Household,( shady side ),-
...,...,...,...,...,...,...,...,...
46650,9f4a33b1-236c-414b-b7a9-c0d24b500b20,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+
46651,a390eeaf-7d6c-4c93-95e5-32ed1dec60da,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+
46652,b9ff9176-a1ad-4ce5-87e8-6640be3aded7,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+
46653,c2d80cdc-d62c-450a-a6c8-90ba437e962d,web,( -628032056789376465 ),( Chrome ),( wifi ),( Phone ),( hendersonville ),Unknown Generic Android Tablet


In [69]:
metadata = metadata.merge(vid_nb_of_users, how='left', on='id')
metadata = metadata.merge(vid_nb_of_housheolds, how='left', on='id')

metadata = metadata.merge(user_nb_of_vids, how='left', on='id')
metadata = metadata.merge(user_nb_of_housheolds, how='left', on='id')

metadata = metadata.merge(household_nb_of_vids, how='left', on='id')
metadata = metadata.merge(household_nb_of_users, how='left', on='id')

metadata = metadata.rename(columns={'ua_devices': 'node_type'})
metadata

,id,env,ip_hash,browsers,connection_type,node_type,cities,standardised_name,vid_nb_of_users,vid_nb_of_households,user_nb_of_vids,user_nb_of_households,household_nb_of_vids,household_nb_of_users
0,-9200385283866435985,Household,( 4347902281762601649 ),-,-,Household,( memphis ),-,NaN,NaN,NaN,NaN,3.0,1.0
1,-9192807107845688163,Household,( 3925892373630619020 ),-,-,Household,( johnstown ),-,NaN,NaN,NaN,NaN,7.0,NaN
2,-9190073361424575441,Household,( -6351207366429801774 ),-,-,Household,( ft lee ),-,NaN,NaN,NaN,NaN,3.0,NaN
3,-9189549264388806478,Household,( 2816271962527781015 ),-,-,Household,( columbus ),-,NaN,NaN,NaN,NaN,5.0,NaN
4,-9183468325073019856,Household,( -5157660449457695196 ),-,-,Household,( shady side ),-,NaN,NaN,NaN,NaN,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46650,9f4a33b1-236c-414b-b7a9-c0d24b500b20,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+,0.0,1.0,NaN,NaN,NaN,NaN
46651,a390eeaf-7d6c-4c93-95e5-32ed1dec60da,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+,0.0,1.0,NaN,NaN,NaN,NaN
46652,b9ff9176-a1ad-4ce5-87e8-6640be3aded7,web,( -628032056789376465 ),( Android WebView ),( wifi ),( Tablet ),( hendersonville ),Samsung Galaxy Tab A9+,0.0,1.0,NaN,NaN,NaN,NaN
46653,c2d80cdc-d62c-450a-a6c8-90ba437e962d,web,( -628032056789376465 ),( Chrome ),( wifi ),( Phone ),( hendersonville ),Unknown Generic Android Tablet,0.0,1.0,NaN,NaN,NaN,NaN


In [70]:
edges.to_csv('/Users/ivan-can.arisoy/Desktop/omnichannel_graph/src/data/US Lotame 2001/edges-Lotame-2001.csv', index=False)
metadata.to_csv('/Users/ivan-can.arisoy/Desktop/omnichannel_graph/src/data/US Lotame 2001/metadata-Lotame-2001.csv', index=False)

In [70]:
# 1. Calculate the number of unique households associated with each VID and shared ID
household_counts = edges[edges['relationship'] == 'Household-User'].groupby('target')['source'].nunique().reset_index()
household_counts.columns = ['id', 'nb_of_households']

# 2. Calculate the number of unique users associated with each household
user_counts = edges[edges['relationship'] == 'Household-User'].groupby('source')['target'].nunique().reset_index()
user_counts.columns = ['id', 'nb_of_users']

# 3. Calculate the number of unique VIDs associated with each user
vid_counts = edges[edges['relationship'] == 'User-Device'].groupby('source')['target'].nunique().reset_index()
vid_counts.columns = ['id', 'nb_of_vids']

# Merge these calculated values into the metadata DataFrame
metadata = metadata.merge(household_counts, how='left', on='id')
metadata = metadata.merge(user_counts, how='left', on='id')
metadata = metadata.merge(vid_counts, how='left', on='id')

# Fill any NaN values with 0, as nodes without associations will have NaN after the merge
metadata[['nb_of_households', 'nb_of_users', 'nb_of_vids']] = metadata[['nb_of_households', 'nb_of_users', 'nb_of_vids']].fillna(0)
metadata = metadata.rename(columns={'ua_devices': 'node_type'})